<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [ ]:
# mount drive

In [ ]:
# import CONLL2003 annotated dataset

In [ ]:
# split into train and test

## Spacy NER

In [2]:
# create dir to place spacy ner data
!mkdir spacy_ner_data

# convert CONLL2003 annotation data into spacy data
!python -m spacy convert "train.txt" spacy_ner_data -c ner
!python -m spacy convert "test.txt" spacy_ner_data -c ner
!python -m spacy convert "valid.txt" spacy_ner_data -c ner

sample_data


In [ ]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [ ]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
# train model using hyperparameters set in config.cfg
# trained model in output/ dir
!python -m spacy train config.cfg --output ./output

In [ ]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate output/model-best spacy_ner_data/test.spacy -dp result

In [ ]:
# download trained model

## Flair NER

In [ ]:
!pip install flair

In [ ]:
# load data
import flair
from flair.datasets import ColumnCorpus

# Define columns
columns = {0: 'text', 1: 'ner'}

# Specify the path to your training and test data
data_folder = 'path/to/your/data'  # Update this path
train_file = 'train.txt'  # Your training file
test_file = 'test.txt'    # Your testing file

# Create the corpus
corpus = ColumnCorpus(data_folder,
                      { 'train': train_file,
                        'test': test_file },
                      columns=columns)

In [ ]:
# create NER tagger
from flair.models import SequenceTagger

tagger = SequenceTagger(hidden_size=256,
                         embeddings='glove',
                         tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),
                         tag_type='ner',
                         use_crf=True)


In [ ]:
# train model
from flair.trainers import ModelTrainer

trainer = ModelTrainer(tagger, corpus)

trainer.train('path/to/save/model',  # Update this path
               learning_rate=0.1,
               mini_batch_size=32,
               max_epochs=10)

In [ ]:
# evaluate model
result, score = trainer.evaluate(corpus.test)
print(result)
print(score)

In [ ]:
# make prediction
model = SequenceTagger.load('path/to/save/model')
sentence = flair.data.Sentence("Your text here.")

model.predict(sentence)

print(sentence.to_tagged_string())